In [1]:
from tensorflow import keras
import pickle
import tensorflow as tf
import keras_nlp
MAX_SEQUENCE_LENGTH = 40

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# Load the model
loaded_model = keras.models.load_model("transformer_model")

# Load jw_vocab
with open('jw_vocab.pkl', 'rb') as f:
    jw_vocab = pickle.load(f)

# Load rm_vocab
with open('rm_vocab.pkl', 'rb') as f:
    rm_vocab = pickle.load(f)

In [3]:
jw_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=jw_vocab, lowercase=False
)
rm_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=rm_vocab, lowercase=False
)

In [4]:
def decode_sequences(input_sentences):
    batch_size = tf.shape(input_sentences)[0]

    # Tokenize the encoder input.
    encoder_input_tokens = jw_tokenizer(input_sentences).to_tensor(
        shape=(None, MAX_SEQUENCE_LENGTH)
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def next(prompt, cache, index):
        logits = loaded_model([encoder_input_tokens, prompt])[:, index - 1, :]
        # Ignore hidden states for now; only needed for contrastive search.
        hidden_states = None
        return logits, hidden_states, cache

    # Build a prompt of length 40 with a start token and padding tokens.
    length = 40
    start = tf.fill((batch_size, 1), rm_tokenizer.token_to_id("[START]"))
    pad = tf.fill((batch_size, length - 1), rm_tokenizer.token_to_id("[PAD]"))
    prompt = tf.concat((start, pad), axis=-1)

    generated_tokens = keras_nlp.samplers.GreedySampler()(
        next,
        prompt,
        end_token_id=rm_tokenizer.token_to_id("[END]"),
        index=1,  # Start sampling after start token.
    )
    generated_sentences = rm_tokenizer.detokenize(generated_tokens)
    return generated_sentences

In [5]:
# input_sentence = 'ساي ساكيت ڤروت'
# input_sentence = 'ساي هاڽ ڬاديس كامڤوڠ.'
# input_sentence = 'اوق دڠكي'
input_sentence = 'saya suka kamu'
translated = decode_sequences(tf.constant([input_sentence]))
translated = translated.numpy()[0].decode("utf-8")
translated = (
    translated.replace("[PAD]", "")
    .replace("[START]", "")
    .replace("[END]", "")
    .strip()
)

print(input_sentence)
print(translated)

saya suka kamu
nya jan


In [1]:
import tensorflow as tf

print(tf.keras.__version__)

import keras

print(keras.__version__)


2.10.0
2.10.0
